# 01 - Data ExtractionExtract transaction data from regional Excel files.

## Setup

In [ ]:
import pandas as pdimport numpy as npfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')# PathsBASE_PATH = Path('..')RAW_DATA_PATH = BASE_PATH / '2025'OUTPUT_PATH = BASE_PATH / 'features_v2'OUTPUT_PATH.mkdir(exist_ok=True)print(f"Raw data path: {RAW_DATA_PATH}")print(f"Output path: {OUTPUT_PATH}")

## Find all Excel files

In [ ]:
# Find all Excel filesexcel_files = list(RAW_DATA_PATH.glob('**/*.xlsx'))print(f"Found {len(excel_files)} Excel files")# Group by monthby_month = {}for f in excel_files:    month = f.parent.name    if month not in by_month:        by_month[month] = []    by_month[month].append(f)for month, files in sorted(by_month.items()):    print(f"  {month}: {len(files)} files")

## Extract Region from Filename

In [ ]:
REGIONS = {    'Cape Town': 'Cape Town',    'Gauteng': 'Gauteng',     'Capital Gauteng': 'Gauteng',    'Polokwane': 'Polokwane',    'George': 'George',    'Hardware': 'Hardware'}def extract_region(filename):    for pattern, region in REGIONS.items():        if pattern.lower() in filename.lower():            return region    return 'Unknown'# Testfor f in excel_files[:5]:    print(f"{f.name} -> {extract_region(f.name)}")

## Extract Line Items from Excel

In [ ]:
def extract_line_items(excel_path):    """Extract all line items from an Excel file."""    all_items = []    region = extract_region(excel_path.name)        try:        xl = pd.ExcelFile(excel_path)                for sheet in xl.sheet_names:            # Skip summary/master sheets            if any(x in sheet.lower() for x in ['summary', 'debtor', 'master']):                continue                        try:                df = pd.read_excel(xl, sheet_name=sheet)                                # Find SKU column                sku_col = None                for col in df.columns:                    if 'sku' in str(col).lower() or 'code' in str(col).lower():                        sku_col = col                        break                                if sku_col is None:                    continue                                # Find quantity and price columns                qty_col = next((c for c in df.columns if 'qty' in str(c).lower() or 'quantity' in str(c).lower()), None)                price_col = next((c for c in df.columns if 'price' in str(c).lower() or 'unit' in str(c).lower()), None)                total_col = next((c for c in df.columns if 'total' in str(c).lower() or 'amount' in str(c).lower()), None)                                # Extract rows                for _, row in df.iterrows():                    sku = row.get(sku_col)                    if pd.isna(sku) or str(sku).strip() == '':                        continue                                        all_items.append({                        'invoice_id': sheet,                        'sku': str(sku).strip(),                        'quantity': row.get(qty_col, 0) if qty_col else 0,                        'unit_price': row.get(price_col, 0) if price_col else 0,                        'line_total': row.get(total_col, 0) if total_col else 0,                        'region': region,                        'source_file': excel_path.name                    })            except Exception as e:                continue                    except Exception as e:        print(f"Error reading {excel_path.name}: {e}")        return all_items# Test on one fileif excel_files:    test_items = extract_line_items(excel_files[0])    print(f"Extracted {len(test_items)} items from {excel_files[0].name}")

## Extract All Files

In [ ]:
# Extract all files (this takes a few minutes)all_line_items = []for i, excel_file in enumerate(excel_files):    items = extract_line_items(excel_file)    all_line_items.extend(items)        if (i + 1) % 10 == 0:        print(f"Processed {i + 1}/{len(excel_files)} files, {len(all_line_items)} items so far")print(f"\nTotal items extracted: {len(all_line_items)}")# Create DataFramedf = pd.DataFrame(all_line_items)print(f"Columns: {df.columns.tolist()}")df.head()

## Save Raw Extraction

In [ ]:
# Save raw extractiondf.to_csv(OUTPUT_PATH / 'raw_extraction.csv', index=False)print(f"✓ Saved: {OUTPUT_PATH / 'raw_extraction.csv'}")print(f"  Rows: {len(df):,}")print(f"  Unique SKUs: {df['sku'].nunique():,}")print(f"  Regions: {df['region'].unique().tolist()}")